In [ ]:
# Import packages needed to create a imaage classification model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Please replace the brackets below with the drive location of your folder which included subfolders for images
# Sample path: /content/drive/My Drive/ImageClassification
PATH = '/content/drive/My Drive/NebulasWithoutDarkReflection'

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
# Link from which the dataset is downloaded

train_dir = os.path.join(PATH)
validation_dir = os.path.join(PATH)

BATCH_SIZE = 45
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir, 
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 232 files belonging to 3 classes.


In [ ]:
# Create the data generation pipeline for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2) # set validation split




train_generator = train_datagen.flow_from_directory(PATH, 
                                                 target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                  subset = 'training')
validation_generator = train_datagen.flow_from_directory(PATH,
                                                 target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='categorical',
                                                 shuffle=False,
                                                  subset = 'validation')

Found 187 images belonging to 3 classes.
Found 45 images belonging to 3 classes.


In [ ]:
# Download the efs model
base_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
#resnet50.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
  layer.trainable=False
  
"""
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in base_model.layers[:185]:
    layer.trainable=False
for layer in base_model.layers[185:]:
    layer.trainable=True
"""

# Specify the number of classes
# This is again hardcoded for this experiment
num_classes = 3

# Create the base model
model = create_model(base_model,num_classes)

print(len(base_model.layers))


190


In [ ]:
base_learning_rate = 0.001 #decrease for different results; use excel sheet to note down results from each change to learning rate and epochs
opt = tf.optimizers.Adam(learning_rate=base_learning_rate)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
history_fine = model.fit(train_generator,
                         steps_per_epoch=step_size_train,
                         epochs=50, #<-- increase for higher accuracy
                         validation_data = validation_generator)

Epoch 1/50
4/4 [==============================] - 10s 2s/step - loss: 268.0345 - accuracy: 0.4778 - val_loss: 134.7354 - val_accuracy: 0.3556
Epoch 2/50
4/4 [==============================] - 6s 1s/step - loss: 138.9775 - accuracy: 0.4085 - val_loss: 19.0081 - val_accuracy: 0.6222
Epoch 3/50
4/4 [==============================] - 6s 2s/step - loss: 111.6407 - accuracy: 0.6127 - val_loss: 83.2561 - val_accuracy: 0.4444
Epoch 4/50
4/4 [==============================] - 6s 2s/step - loss: 44.0038 - accuracy: 0.6444 - val_loss: 84.4555 - val_accuracy: 0.4667
Epoch 5/50
4/4 [==============================] - 6s 2s/step - loss: 42.8770 - accuracy: 0.5556 - val_loss: 25.7622 - val_accuracy: 0.5778
Epoch 6/50
4/4 [==============================] - 6s 2s/step - loss: 25.7225 - accuracy: 0.5000 - val_loss: 38.8838 - val_accuracy: 0.5778
Epoch 7/50
4/4 [==============================] - 6s 2s/step - loss: 23.7315 - accuracy: 0.6944 - val_loss: 40.0266 - val_accuracy: 0.6000
Epoch 8/50
4/4 [======

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#Confution Matrix and Classification Report
#Confusion Matrix to show accuracy of each bucket
Y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Emission', 'Planetary', 'Supernova Remnants']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix
[[1 2 3 1 4]
 [0 3 4 0 4]
 [0 3 9 0 1]
 [1 5 4 0 2]
 [2 3 3 3 4]]
Classification Report
                    precision    recall  f1-score   support

              Dark       0.25      0.09      0.13        11
          Emission       0.19      0.27      0.22        11
         Planetary       0.39      0.69      0.50        13
        Reflection       0.00      0.00      0.00        12
Supernova Remnants       0.27      0.27      0.27        15

          accuracy                           0.27        62
         macro avg       0.22      0.26      0.22        62
      weighted avg       0.22      0.27      0.23        62



'\ndark - 61\nreflection - 66\nplanetary - 68\nemission - 59\nsupernova remnants - 76\n'

In [ ]:
print (train_generator.class_indices)


{'Dark': 0, 'Emission': 1, 'Planetary': 2, 'Reflection': 3, 'Supernova Remnants': 4}


In [ ]:
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

print('Training Accuracy: ', acc)
print('Validation Accuracy: ', acc)
print('Training Loss: ', loss)
print('Validation Loss: ', val_loss)

In [ ]:
def create_model(base_model, num_classes):
    # Grab the last layer and add a few extra layers to it
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    # Dense layer 1
    x=tf.keras.layers.Dense(1024,activation='relu', kernel_initializer=tf.keras.initializers.VarianceScaling, use_bias=True)(x)

    #Final layer with softmax activation
    preds=tf.keras.layers.Dense(num_classes,activation='softmax', kernel_initializer=tf.keras.initializers.VarianceScaling, use_bias=False)(x) 
    
    # Create the final model
    model=Model(inputs=base_model.input,outputs=preds)
    return model